In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')
test_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 104) (3492890, 103)
CPU times: user 2min 24s, sys: 8.5 s, total: 2min 32s
Wall time: 27.9 s


In [5]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 3min 49s, sys: 13.6 s, total: 4min 2s
Wall time: 46.2 s


In [7]:
most_imp_ones2 = ['A1_minus_3_sigma','A5_max_median_diff_flux','A5_minus_3_sigma','A5_max_mean_diff_flux']
train_metadata_kaggle.drop(most_imp_ones2,axis=1,inplace=True)
test_metadata_kaggle.drop(most_imp_ones2,axis=1,inplace=True)
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 100) (3492890, 99)


In [8]:
%%time
test_metadata_kaggle = test_metadata_kaggle.reset_index(drop=True)
test_metadata = test_metadata.reset_index(drop=True)

CPU times: user 4.45 s, sys: 6.82 s, total: 11.3 s
Wall time: 11.2 s


In [9]:
print(train_metadata_kaggle['object_id'].equals(train_metadata['object_id']))

True


In [10]:
print(test_metadata_kaggle['object_id'].equals(test_metadata['object_id']))

True


In [11]:
test_metadata_kaggle.tail()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF
3492885,130787966,-49.490440,75.235832,3.905430,1.703813,21.009541,0.625654,1.341227,70.066055,14.846084,10.951475,12.628468,1.307980,0.013793,374.420803,8.678639,5.954086e+03,7.560569,15.902124,124.726272,31.936629,7.843372,35.655183,32.200453,0.154555,-0.174445,23.565507,13.595040,6.803259,2.380728,63.739853,39.279845,3.023876,1.647382,82.545904,81.181195,0.670890,0.882368,187.846479,74.176357,3.792912,1.576256,172.934431,184.079710,-0.297825,0.291885,145.0,8.0,10.0,20.061909,0.151706,4.0,43.0,89.761400,89.761400,89.7614,0.4493,0.9954,41.9836,0.036,1.937777,2.277911,1.215719,35.655180,2.377012,0.0000,0.0909,0.0,0.0,0.000,0.00000,0.5557,-2.620740,0.0,5.474970e-13,0.103244,2.16581,0.050748,0.683935,2.26490,0.829141,2.439200,2.16581,0.500000,0.953504,0.808853,0.966782,2.32229,0.185745,19.47180,0.427032,10.79380,0.000000,1.000000,1.000000,0,0,0,0
3492886,130787971,-58.301781,59.105938,2.910128,1.390304,18.432931,-0.033138,1.598772,70.469521,14.704315,10.958591,12.573206,1.570236,0.014286,772.852614,9.258595,2.735284e+04,9.229165,35.392055,117.407719,40.344517,3.317347,47.270954,21.592998,-1.528033,0.106061,23.940401,25.706965,1.965597,0.964946,106.555644,97.317107,4.782199,2.138468,13.695812,30.341361,-0.615031,-0.353402,198.983121,128.149311,1.826418,0.809600,111.513930,121.261346,-0.765793,-0.401619,140.0,6.0,10.0,16.961143,-0.250912,5.0,95.0,0.977400,0.977400,0.9774,0.6729,0.0614,43.0419,0.083,2.063756,0.297612,0.715511,-47.270954,-3.376497,0.0000,0.0000,2.0,0.0,0.000,0.00000,0.6960,-13.268991,0.0,8.997960e-14,0.009136,3.61916,-0.781019,0.920752,2.63034,0.692992,0.862991,3.61916,0.652174,0.912471,0.714979,-0.248684,1.33370,0.073276,16.81010,0.711031,28.97450,0.000000,1.000000,1.000000,0,0,0,0
3492887,130787974,-79.991745,341.318909,45.861197,3.651273,99.890188,1.724598,2.153563,62.515724,17.602066,12.978883,13.797900,1.212612,0.176056,19850.226004,4.094035,5.000366e+06,4.974861,251.9047

In [12]:
test_metadata.tail()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_plus_1,pe

In [13]:
train_metadata_kaggle.tail()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF
7843,130739978,-105.375282,517.602478,6.786007,1.599034,50.600944,7.259594,1.700991,72.230759,16.391897,11.132936,14.645405,1.401451,0.034014,3178.336379,11.992239,4.882531e+05,11.679274,153.619074,622.977760,91.803294,4.055341,21.873135,28.266883,5.705815,1.784807,162.456175,162.296936,13.667479,3.680811,35.689574,7.912341,8.977344,2.753125,117.413905,65.508160,-0.460370,0.410709,42.460987,115.925375,0.845991,0.431793,702.571174,302.817480,20.652438,3.998852,147.0,4.0,15.0,32.225018,0.053633,1.0,97.0,199.953250,199.953250,799.8130,0.0000,0.0000,NaN,0.013,65,1.568287,3.671722,0.000000,21.873135,1.286655,0.2426,0.07140,2,0.0000,0.1691,0.02856,0.6250,-7.383,1,2.351330e-29,1.100730e-01,3.12249,1.669840,0.413225,2.63550,0.698985,11.699300,3.12249,0.666667,0.961661,0.881326,0.257244,5.264420,1.573640,16.47190,0.527594,22.95970,0.000,1.0,1.0,0,0,0,0
7844,130755807,-69.036392,363.402466,16.466672,1.489275,68.780504,3.818545,2.326328,71.215874,18.477200,15.121149,14.085974,1.175512,0.052632,3025.366219,5.549463,9.152266e+05,5.798803,302.517634,432.438858,26.261461,1.429467,295.622920,333.272992,10.607947,3.192929,76.624077,61.855298,9.208880,2.970945,14.033524,21.132148,-0.523883,0.014138,380.065039,328.717896,13.153053,3.576847,1018.956828,973.270337,5.009929,2.493254,119.965253,184.278912,2.815352,1.133398,114.0,11.0,23.0,29.287346,0.036457,5.0,55.0,7.403260,7.403260,37.0163,2.5606,1.1146,46.6108,0.136,90,1.719466,-0.907001,7.805601,295.622920,24.635242,0.2886,0.09090,0,0.1200,0.0000,0.00000,0.3890,-9.390,0,1.676440e-09,5.079140e-02,3.74433,2.913970,0.563771,2.49580,1.011080,3.306760,3.74433,0.222222,0.979527,0.790144,4.068030,2.823260,-0.000683,106.13100,0.885791,8.84614,0.000,1.0,1.0,0,0,0,0
7845,130762946,-135.602631,169.916672,-15.308645,-14.336030,35.248387,0.713889,5.974091,79.265930,27.723353,22.661341,19.148498,1.065352,0.052239,706.392995,5.975417,-3.460382e+04,-6.150094,-48.98

In [14]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [15]:
#A0
train_metadata['A0_max_min_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_max_mean_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_max_median_diff_flux'] = train_metadata['A0_max_flux'] - train_metadata['A0_median_flux']
train_metadata['A0_median_mean_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_mean_flux']
train_metadata['A0_median_min_diff_flux'] = train_metadata['A0_median_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_mean_min_diff_flux'] = train_metadata['A0_mean_flux'] - train_metadata['A0_min_flux']
train_metadata['A0_minus_3_sigma'] = train_metadata['A0_mean_flux'] - 3*train_metadata['A0_std_flux']
train_metadata['A0_minus_2_sigma'] = train_metadata['A0_mean_flux'] - 2*train_metadata['A0_std_flux']
train_metadata['A0_minus_1_sigma'] = train_metadata['A0_mean_flux'] - 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_1_sigma'] = train_metadata['A0_mean_flux'] + 1*train_metadata['A0_std_flux']
train_metadata['A0_plus_2_sigma'] = train_metadata['A0_mean_flux'] + 2*train_metadata['A0_std_flux']
train_metadata['A0_plus_3_sigma'] = train_metadata['A0_mean_flux'] + 3*train_metadata['A0_std_flux']
#A1
train_metadata['A1_max_min_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_max_mean_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_max_median_diff_flux'] = train_metadata['A1_max_flux'] - train_metadata['A1_median_flux']
train_metadata['A1_median_mean_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_mean_flux']
train_metadata['A1_median_min_diff_flux'] = train_metadata['A1_median_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_mean_min_diff_flux'] = train_metadata['A1_mean_flux'] - train_metadata['A1_min_flux']
train_metadata['A1_minus_3_sigma'] = train_metadata['A1_mean_flux'] - 3*train_metadata['A1_std_flux']
train_metadata['A1_minus_2_sigma'] = train_metadata['A1_mean_flux'] - 2*train_metadata['A1_std_flux']
train_metadata['A1_minus_1_sigma'] = train_metadata['A1_mean_flux'] - 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_1_sigma'] = train_metadata['A1_mean_flux'] + 1*train_metadata['A1_std_flux']
train_metadata['A1_plus_2_sigma'] = train_metadata['A1_mean_flux'] + 2*train_metadata['A1_std_flux']
train_metadata['A1_plus_3_sigma'] = train_metadata['A1_mean_flux'] + 3*train_metadata['A1_std_flux']
#A2
train_metadata['A2_max_min_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_max_mean_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_max_median_diff_flux'] = train_metadata['A2_max_flux'] - train_metadata['A2_median_flux']
train_metadata['A2_median_mean_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_mean_flux']
train_metadata['A2_median_min_diff_flux'] = train_metadata['A2_median_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_mean_min_diff_flux'] = train_metadata['A2_mean_flux'] - train_metadata['A2_min_flux']
train_metadata['A2_minus_3_sigma'] = train_metadata['A2_mean_flux'] - 3*train_metadata['A2_std_flux']
train_metadata['A2_minus_2_sigma'] = train_metadata['A2_mean_flux'] - 2*train_metadata['A2_std_flux']
train_metadata['A2_minus_1_sigma'] = train_metadata['A2_mean_flux'] - 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_1_sigma'] = train_metadata['A2_mean_flux'] + 1*train_metadata['A2_std_flux']
train_metadata['A2_plus_2_sigma'] = train_metadata['A2_mean_flux'] + 2*train_metadata['A2_std_flux']
train_metadata['A2_plus_3_sigma'] = train_metadata['A2_mean_flux'] + 3*train_metadata['A2_std_flux']
#A3
train_metadata['A3_max_min_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_max_mean_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_max_median_diff_flux'] = train_metadata['A3_max_flux'] - train_metadata['A3_median_flux']
train_metadata['A3_median_mean_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_mean_flux']
train_metadata['A3_median_min_diff_flux'] = train_metadata['A3_median_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_mean_min_diff_flux'] = train_metadata['A3_mean_flux'] - train_metadata['A3_min_flux']
train_metadata['A3_minus_3_sigma'] = train_metadata['A3_mean_flux'] - 3*train_metadata['A3_std_flux']
train_metadata['A3_minus_2_sigma'] = train_metadata['A3_mean_flux'] - 2*train_metadata['A3_std_flux']
train_metadata['A3_minus_1_sigma'] = train_metadata['A3_mean_flux'] - 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_1_sigma'] = train_metadata['A3_mean_flux'] + 1*train_metadata['A3_std_flux']
train_metadata['A3_plus_2_sigma'] = train_metadata['A3_mean_flux'] + 2*train_metadata['A3_std_flux']
train_metadata['A3_plus_3_sigma'] = train_metadata['A3_mean_flux'] + 3*train_metadata['A3_std_flux']
#A4
train_metadata['A4_max_min_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_max_mean_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_max_median_diff_flux'] = train_metadata['A4_max_flux'] - train_metadata['A4_median_flux']
train_metadata['A4_median_mean_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_mean_flux']
train_metadata['A4_median_min_diff_flux'] = train_metadata['A4_median_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_mean_min_diff_flux'] = train_metadata['A4_mean_flux'] - train_metadata['A4_min_flux']
train_metadata['A4_minus_3_sigma'] = train_metadata['A4_mean_flux'] - 3*train_metadata['A4_std_flux']
train_metadata['A4_minus_2_sigma'] = train_metadata['A4_mean_flux'] - 2*train_metadata['A4_std_flux']
train_metadata['A4_minus_1_sigma'] = train_metadata['A4_mean_flux'] - 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_1_sigma'] = train_metadata['A4_mean_flux'] + 1*train_metadata['A4_std_flux']
train_metadata['A4_plus_2_sigma'] = train_metadata['A4_mean_flux'] + 2*train_metadata['A4_std_flux']
train_metadata['A4_plus_3_sigma'] = train_metadata['A4_mean_flux'] + 3*train_metadata['A4_std_flux']
#A5
train_metadata['A5_max_min_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_max_mean_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_max_median_diff_flux'] = train_metadata['A5_max_flux'] - train_metadata['A5_median_flux']
train_metadata['A5_median_mean_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_mean_flux']
train_metadata['A5_median_min_diff_flux'] = train_metadata['A5_median_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_mean_min_diff_flux'] = train_metadata['A5_mean_flux'] - train_metadata['A5_min_flux']
train_metadata['A5_minus_3_sigma'] = train_metadata['A5_mean_flux'] - 3*train_metadata['A5_std_flux']
train_metadata['A5_minus_2_sigma'] = train_metadata['A5_mean_flux'] - 2*train_metadata['A5_std_flux']
train_metadata['A5_minus_1_sigma'] = train_metadata['A5_mean_flux'] - 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_1_sigma'] = train_metadata['A5_mean_flux'] + 1*train_metadata['A5_std_flux']
train_metadata['A5_plus_2_sigma'] = train_metadata['A5_mean_flux'] + 2*train_metadata['A5_std_flux']
train_metadata['A5_plus_3_sigma'] = train_metadata['A5_mean_flux'] + 3*train_metadata['A5_std_flux']

In [16]:
print(train_metadata.shape)

(7848, 305)


In [17]:
train_metadata.iloc[:,233:].head()

,A0_max_min_diff_flux,A0_max_mean_diff_flux,A0_max_median_diff_flux,A0_median_mean_diff_flux,A0_median_min_diff_flux,A0_mean_min_diff_flux,A0_minus_3_sigma,A0_minus_2_sigma,A0_minus_1_sigma,A0_plus_1_sigma,A0_plus_2_sigma,A0_plus_3_sigma,A1_max_min_diff_flux,A1_max_mean_diff_flux,A1_max_median_diff_flux,A1_median_mean_diff_flux,A1_median_min_diff_flux,A1_mean_min_diff_flux,A1_minus_3_sigma,A1_minus_2_sigma,A1_minus_1_sigma,A1_plus_1_sigma,A1_plus_2_sigma,A1_plus_3_sigma,A2_max_min_diff_flux,A2_max_mean_diff_flux,A2_max_median_diff_flux,A2_median_mean_diff_flux,A2_median_min_diff_flux,A2_mean_min_diff_flux,A2_minus_3_sigma,A2_minus_2_sigma,A2_minus_1_sigma,A2_plus_1_sigma,A2_plus_2_sigma,A2_plus_3_sigma,A3_max_min_diff_flux,A3_max_mean_diff_flux,A3_max_median_diff_flux,A3_median_mean_diff_flux,A3_median_min_diff_flux,A3_mean_min_diff_flux,A3_minus_3_sigma,A3_minus_2_sigma,A3_minus_1_sigma,A3_plus_1_sigma,A3_plus_2_sigma,A3_plus_3_sigma,A4_max_min_diff_flux,A4_max_mean_diff_flux,A4_max_median_diff_flux,A4_median_mean_diff_flux,A4_median_min_diff_flux,A4_mean_min_diff_flux,A4_minus_3_sigma,A4_minus_2_sigma,A4_minus_1_sigma,A4_plus_1_sigma,A4_plus_2_sigma,A4_plus_3_sigma,A5_max_min_diff_flux,A5_max_mean_diff_flux,A5_max_median_diff_flux,A5_median_mean_diff_flux,A5_median_min_diff_flux,A5_mean_min_diff_flux,A5_minus_3_sigma,A5_minus_2_sigma,A5_minus_1_sigma,A5_plus_1_sigma,A5_plus_2_sigma,A5_plus_3_sigma
0,242.096030,128.437365,135.198035,-6.760670,106.897995,113.658665,-255.088745,-171.144015,-87.199285,80.690176,164.634906,248.579636,1760.626340,1046.426340,1148.626340,-102.2000,612.0000,714.2000,-2191.161900,-1589.374600,-987.587300,215.987300,817.774600,1419.561900,1293.984560,746.131120,877.784560,-131.653440,416.2000,547.853440,-1499.510580,-1044.389240,-589.267900,320.974780,776.096120,1231.217460,976.237060,566.840560,607.837060,-40.996500,368.4000,409.396500,-1127.378650,-791.953600,-456.528550,214.321550,549.746600,885.171650,804.138240,437.908334,485.495110,-47.586776,318.643130,366.229906,-931.364914,-639.561474,-347.758034,235.848846,527.652286,819.455726,801.003240,425.637990,463.712450,-38.074460,337.290790,375.365250,-931.788350,-637.008850,-342.229350,247.329650,542.109150,836.888650
1,29.245007,17.230227,17.606634,-0.376407,11.638373,12.014780,-24.060925,-16.947416,-9.833907,4.393111,11.506620,18.620129,20.849021,10.149021,9.690521,0.4585,11.1585,10.7000,-18.157002,-12.444668,-6.732334,4.692334,10.404668,16.117002,20.599041,11.323279,10.647041,0.676238,9.9520,9.275762,-18.106451,-12.335713,-6.564976,4.976500,10.747237,16.517975,23.730316,12.317282,11.404216,0.913066,12.3261,11.413034,-20.338205,-13.887792,-7.437379,5.463447,11.913860,18.364273,22.114735,10.728195,10.620110,0.108086,11.494625,11.386539,-20.121229,-13.714239,-7.307251,5.506728,11.913717,18.320706,28.982050,16.565061,17.233897,-0.668837,11.748153,12.416989,-23.076394,-15.982321,-8.888248,5.299898,12.393971,19.488044
2,9.402126,5.990246,5.918073,0.072173,3.484053,3.411880,-5.534697,-3.705825,-1.876953,1.780792,3.609664,5.438536,9.086109,5.552009,5.521709,0.0303,3.5644,3.5341,-5.280586,-3.473357,-1.666129,1.948329,3.755557,5.562786,23.844710,18.593840,20.503010,-1.909170,3.3417,5.250870,-14.277579,-8.718096,-3.158613,7.960353,13.519836,19.079319,39.010100,30.335936,32.911500,-2.575564,6.0986,8.674164,-21.339797,-13.147810,-4.955823,11.428151,19.620138,27.812125,46.996290,36.851252,40.155626,-3.304374,6.840664,10.145038,-27.822304,-17.111960,-6.401616,15.019072,25.729416,36.439760,66.469871,42.770664,44.767413,-1.996749,21.702458,23.699207,-35.458878,-22.126120,-8.793362,17.872154,31.204912,44.537670
3,21.888378,16.216506,16.957315,-0.740809,4.931063,5.671872,-11.325812,-6.951367,-2.576922,6.171968,10.546413,14.920858,195.864300,186.524300,191.356300,-4.8320,4.5080,9.3400,-72.173977,-46.209318,-20.244659,31.684659,57.649318,83.613977,222.955210,211.083678,220.370910,-9.287232,2.5843,11.871532,-86.162462,-54.204464,-22.246466,41.669530,73.627528,105.5855

In [18]:
print(train_metadata[['object_id'] + list(train_metadata.columns)[233:]].shape)

(7848, 73)


In [19]:
print(train_metadata_kaggle.shape)

(7848, 100)


In [ ]:
"""%%time
train_metadata_kaggle = train_metadata_kaggle.merge(train_metadata[['object_id'] + list(train_metadata.columns)[233:]],
                                                   how='left',on = 'object_id')"""

In [ ]:
"""%%time
f1_train = dd.read_csv('cesium_train_embeddings.csv')
f1_test = dd.read_csv('cesium_test_embeddings.csv')
f1_train = f1_train.compute()
f1_test = f1_test.compute()
print(f1_train.shape,f1_test.shape)"""

In [ ]:
"""%%time
train_metadata_kaggle = train_metadata_kaggle.merge(f1_train,how='left',on = 'object_id')
test_metadata_kaggle = test_metadata_kaggle.merge(f1_test,how='left',on = 'object_id')
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)"""

In [ ]:
"""%%time
f2_train = dd.read_csv('myfeatures_train_embeddings.csv')
f2_test = dd.read_csv('myfeatures_test_embeddings.csv')
f2_train = f2_train.compute()
f2_test = f2_test.compute()
print(f2_train.shape,f2_test.shape)"""

In [ ]:
"""%%time
train_metadata_kaggle = train_metadata_kaggle.merge(f2_train,how='left',on = 'object_id')
test_metadata_kaggle = test_metadata_kaggle.merge(f2_test,how='left',on = 'object_id')
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)"""

In [20]:
test_id = test_metadata_kaggle['object_id']

In [21]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [22]:
"""%%time
cesium_drop = ['__flux_percentile_ratio_mid50___5_', '__flux_percentile_ratio_mid65___2_', '__median_absolute_deviation___2_',
  '__qso_log_chi2_qsonu___0_', '__stetson_k___1_', '__freq1_signif___2_', '__stetson_k___2_', '__freq3_amplitude1___1_',
   '__median_absolute_deviation___2_', '__percent_close_to_median___2_',
   '__freq_varrat___5_','__freq_varrat___4_','__qso_log_chi2_qsonu___3_','__qso_log_chi2_qsonu___1_',
'__qso_log_chi2_qsonu___5_','__std___4_', '__freq_varrat___3_','__amplitude___2_']
columns_from_my_data = ['A0_sum_flux', 'A0_mean_flux', 'A0_std_detected', 'A1_mean_detected', 'A2_sum_detected', 'A4_mean_detected',
 'A5_std_detected', 'A5_mean_detected', 'percent_p2_region_minus_1', 'A2_min_flux', 'A5_sum_detected']
train_metadata_kaggle.drop(cesium_drop,inplace=True,axis=1)
test_metadata_kaggle.drop(cesium_drop,inplace=True,axis=1)
train_metadata_kaggle.drop(columns_from_my_data,inplace=True,axis=1)
test_metadata_kaggle.drop(columns_from_my_data,inplace=True,axis=1)"""

"%%time\ncesium_drop = ['__flux_percentile_ratio_mid50___5_', '__flux_percentile_ratio_mid65___2_', '__median_absolute_deviation___2_',\n  '__qso_log_chi2_qsonu___0_', '__stetson_k___1_', '__freq1_signif___2_', '__stetson_k___2_', '__freq3_amplitude1___1_',\n   '__median_absolute_deviation___2_', '__percent_close_to_median___2_',\n   '__freq_varrat___5_','__freq_varrat___4_','__qso_log_chi2_qsonu___3_','__qso_log_chi2_qsonu___1_',\n'__qso_log_chi2_qsonu___5_','__std___4_', '__freq_varrat___3_','__amplitude___2_']\ncolumns_from_my_data = ['A0_sum_flux', 'A0_mean_flux', 'A0_std_detected', 'A1_mean_detected', 'A2_sum_detected', 'A4_mean_detected',\n 'A5_std_detected', 'A5_mean_detected', 'percent_p2_region_minus_1', 'A2_min_flux', 'A5_sum_detected']\ntrain_metadata_kaggle.drop(cesium_drop,inplace=True,axis=1)\ntest_metadata_kaggle.drop(cesium_drop,inplace=True,axis=1)\ntrain_metadata_kaggle.drop(columns_from_my_data,inplace=True,axis=1)\ntest_metadata_kaggle.drop(columns_from_my_data,inpl

In [23]:
train_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF
0,615,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835268,1.744747,1.623740,0.946023,2.929669e+06,0.812722,-9.601766e+08,-1.414322,-327.742307,1761.066406,-14.306331,-5.373326,205.036926,1628.427737,-1.475181,0.128917,22370.594834,2806.374162,-1.255123,0.415580,7780.500807,2805.598113,-1.409885,0.339918,7024.003068,2536.068846,-1.449858,0.293128,3245.366349,2741.539785,-1.548319,0.200096,2704.641265,2893.344217,-1.592820,0.125268,352.0,19.0,29.0,202.114067,1.999688,35.0,4.0,2.631898,2.631898,873.7903,0.0000,0.0000,NaN,0.017,92,0.319006,-1.528827,0.000000,-205.03693,-3.254554,0.3528,0.9653,57,0.9830,0.2854,0.9120,0.362,-682.000,52,5.562230e-26,6.719410e-20,368.12900,6.217890,1.091730,5.49891,1.053490,114.465000,368.12900,0.172414,0.401664,0.129578,9.16612,9.508750,7.344980,289.27700,0.110785,646.9220,0.000,1.000000,1.0,0,0,0,0
1,713,-14.735178,14.770886,-1.423351,-0.873033,6.471144,0.014989,0.639458,9.115748,2.359620,1.998217,1.509888,1.633246,0.171429,5.886068e+03,3.439423,-2.875087e+04,-3.454554,-4.884564,29.506064,-20.730002,-6.040676,190.427851,299.586559,-1.014003,0.260052,57.109047,192.539229,-1.097170,-0.087865,44.477327,191.057528,-1.188472,-0.022678,55.270113,212.522263,-1.142896,-0.167176,50.414646,203.892482,-1.190245,-0.064134,100.473776,143.963093,-0.797047,0.218182,350.0,50.0,73.0,2.935177,-0.050944,199.0,8.0,14.352571,14.352571,846.8017,1.6267,0.2552,45.4063,0.007,88,1.698939,3.258921,2.099614,-190.42786,-2.720398,0.3525,0.2678,15,0.0893,0.0000,0.0000,0.250,-10.070,0,2.119070e-02,8.243180e-02,5.10035,2.187190,1.066100,3.95669,1.088180,0.851103,5.10035,0.178571,0.369518,0.166179,2.79753,3.124810,0.659762,6.34953,0.111883,10.2985,0.875,1.909016,2.0,1,0,1,1
2,730,-19.159811,47.310059,2.267434,0.409172,8.022239,3.177854,0.695106,11.281384,2.471061,1.990851,1.721134,1.823726,0.069697,4.124452e+03,5.480405,1.046502e+05,5

In [ ]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + temp_list ],on = 'object_id',how = 'left')
print(temp.shape)
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 1000, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10,
'min_child_weight': 200.0, 
#'min_child_weight': 100.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
#'reg_alpha': 0.1,
'reg_alpha': 0.0, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

In [ ]:
imp_df.shape

In [ ]:
imp_df.sort_values('gain',ascending=False).head()

In [ ]:
"""
A0_max_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62026 
A0_max_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.62098 
A0_max_median_diff_flux MULTI WEIGHTED LOG LOSS : 0.61955 
A0_median_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.62035 
A0_median_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62048 
A0_mean_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62152 
A0_minus_3_sigma MULTI WEIGHTED LOG LOSS : 0.61958 
A0_minus_2_sigma MULTI WEIGHTED LOG LOSS : 0.61976 
A0_minus_1_sigma MULTI WEIGHTED LOG LOSS : 0.62056 
A0_plus_1_sigma MULTI WEIGHTED LOG LOSS : 0.62043 
A0_plus_2_sigma MULTI WEIGHTED LOG LOSS : 0.61928 
A0_plus_3_sigma MULTI WEIGHTED LOG LOSS : 0.61954 
A1_max_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.61963 
A1_max_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.61978 
A1_max_median_diff_flux MULTI WEIGHTED LOG LOSS : 0.62008 
A1_median_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.62112 
A1_median_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62213 
A1_mean_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62058 
A1_minus_3_sigma MULTI WEIGHTED LOG LOSS : 0.61839 
A1_minus_2_sigma MULTI WEIGHTED LOG LOSS : 0.61947 
A1_minus_1_sigma MULTI WEIGHTED LOG LOSS : 0.61933 
A1_plus_1_sigma MULTI WEIGHTED LOG LOSS : 0.62118 
A1_plus_2_sigma MULTI WEIGHTED LOG LOSS : 0.61946 
A1_plus_3_sigma MULTI WEIGHTED LOG LOSS : 0.61952 
A2_max_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.61967 
A2_max_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.61972 
A2_max_median_diff_flux MULTI WEIGHTED LOG LOSS : 0.62028 
A2_median_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.62049 
A2_median_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62083 
A2_mean_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.61850 
A2_minus_3_sigma MULTI WEIGHTED LOG LOSS : 0.62067 
A2_minus_2_sigma MULTI WEIGHTED LOG LOSS : 0.62126 
A2_minus_1_sigma MULTI WEIGHTED LOG LOSS : 0.61971 
A2_plus_1_sigma MULTI WEIGHTED LOG LOSS : 0.61973 
A2_plus_2_sigma MULTI WEIGHTED LOG LOSS : 0.62033 
A2_plus_3_sigma MULTI WEIGHTED LOG LOSS : 0.62067 
A3_max_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.61998 
A3_max_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.61997 
A3_max_median_diff_flux MULTI WEIGHTED LOG LOSS : 0.62193 
A3_median_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.62020 
A3_median_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62124 
A3_mean_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62088 
A3_minus_3_sigma MULTI WEIGHTED LOG LOSS : 0.61999 
A3_minus_2_sigma MULTI WEIGHTED LOG LOSS : 0.61978 
A3_minus_1_sigma MULTI WEIGHTED LOG LOSS : 0.62008 
A3_plus_1_sigma MULTI WEIGHTED LOG LOSS : 0.61988 
A3_plus_2_sigma MULTI WEIGHTED LOG LOSS : 0.62052 
A3_plus_3_sigma MULTI WEIGHTED LOG LOSS : 0.62014 
A4_max_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62021 
A4_max_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.61899 
A4_max_median_diff_flux MULTI WEIGHTED LOG LOSS : 0.62017 
A4_median_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.61976 
A4_median_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62030 
A4_mean_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62139 
A4_minus_3_sigma MULTI WEIGHTED LOG LOSS : 0.62063 
A4_minus_2_sigma MULTI WEIGHTED LOG LOSS : 0.62002 
A4_minus_1_sigma MULTI WEIGHTED LOG LOSS : 0.61996 
A4_plus_1_sigma MULTI WEIGHTED LOG LOSS : 0.62018 
A4_plus_2_sigma MULTI WEIGHTED LOG LOSS : 0.62046 
A4_plus_3_sigma MULTI WEIGHTED LOG LOSS : 0.62089 
A5_max_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.61948 
A5_max_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.61894 
A5_max_median_diff_flux MULTI WEIGHTED LOG LOSS : 0.61919 
A5_median_mean_diff_flux MULTI WEIGHTED LOG LOSS : 0.62023 
A5_median_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.62032 
A5_mean_min_diff_flux MULTI WEIGHTED LOG LOSS : 0.61997 
A5_minus_3_sigma MULTI WEIGHTED LOG LOSS : 0.61816 
A5_minus_2_sigma MULTI WEIGHTED LOG LOSS : 0.61926 
A5_minus_1_sigma MULTI WEIGHTED LOG LOSS : 0.61990 
A5_plus_1_sigma MULTI WEIGHTED LOG LOSS : 0.61806 
A5_plus_2_sigma MULTI WEIGHTED LOG LOSS : 0.61976 
A5_plus_3_sigma MULTI WEIGHTED LOG LOSS : 0.61879 
"""

In [ ]:
%%time
final_dict = {}

for column_ in list(train_metadata.columns)[233:]:
    loss_list = []
    temp = train_metadata_kaggle.copy()
    temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
    y = temp['target']
    del temp['target']
    classes = sorted(y.unique())

    class_weight = {
        c: 1 for c in classes
    }
    for c in [64, 15]:
        class_weight[c] = 2

    train_id = temp['object_id']
    del temp['object_id']
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
    'random_state':51,
    'device': 'cpu', 
    'objective': 'multiclass', 
    'num_class': 14, 
    'boosting_type': 'gbdt', 
    'n_jobs': -1, 
    'max_depth': 7, 
    'n_estimators': 1000, 
    'subsample_freq': 2, 
    'subsample_for_bin': 5000, 
    'min_data_per_group': 100, 
    'max_cat_to_onehot': 4, 
    'cat_l2': 1.0, 
    'cat_smooth': 59.5, 
    'max_cat_threshold': 32, 
    'metric_freq': 10, 
    'verbosity': -1, 
    'metric': 'multi_logloss', 
    'xgboost_dart_mode': False, 
    'uniform_drop': False, 
    'colsample_bytree': 0.5, 
    'drop_rate': 0.173, 
    'learning_rate': 0.0267, 
    'max_drop': 5, 
    'min_child_samples': 10,
    'min_child_weight': 200.0, 
    #'min_child_weight': 100.0, 
    'min_split_gain': 0.1, 
    'num_leaves': 7, 
    #'reg_alpha': 0.1,
    'reg_alpha': 0.0, 
    'reg_lambda': 0.00023, 
    'skip_drop': 0.44, 
    'subsample': 0.75}
    oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
        val_x, val_y = temp.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=False,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
        loss_list.append(loss_oof)
        #print(fold_,loss_oof)

        imp_df = pd.DataFrame()
        imp_df['feature'] = temp.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(column_,'MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
    final_dict[column_] = loss_list

In [ ]:
final_dict2 = pd.Series(final_dict)
final_dict2 = pd.DataFrame(final_dict2)
final_dict2['fold1'] = final_dict2[0].apply(lambda x: x[0])
final_dict2['fold2'] = final_dict2[0].apply(lambda x: x[1])
final_dict2['fold3'] = final_dict2[0].apply(lambda x: x[2])
final_dict2['fold4'] = final_dict2[0].apply(lambda x: x[3])
final_dict2['fold5'] = final_dict2[0].apply(lambda x: x[4])

del final_dict2[0]

final_dict2 = final_dict2.reset_index(drop=False)

final_dict2 = final_dict2.rename(columns={'index':'column_name'})

final_dict2['fold1_1'] = (final_dict2['fold1'] > 0.62756) * 1
final_dict2['fold2_1'] = (final_dict2['fold2'] > 0.59905) * 1
final_dict2['fold3_1'] = (final_dict2['fold3'] > 0.65624) * 1
final_dict2['fold4_1'] = (final_dict2['fold4'] > 0.60978) * 1
final_dict2['fold5_1'] = (final_dict2['fold5'] > 0.61012) * 1
final_dict2['fold_sum'] = final_dict2['fold1_1']+ final_dict2['fold2_1'] + final_dict2['fold3_1'] + final_dict2['fold4_1'] + final_dict2['fold5_1']


In [24]:
most_imp_ones = ['A1_minus_3_sigma','A2_mean_min_diff_flux','A4_max_mean_diff_flux','A5_max_mean_diff_flux',
                 'A5_max_median_diff_flux','A5_minus_3_sigma','A5_plus_1_sigma','A5_plus_3_sigma']

In [25]:
most_imp_ones2 = ['A1_minus_3_sigma','A5_max_median_diff_flux','A5_minus_3_sigma','A5_max_mean_diff_flux']

In [26]:
[x for x in most_imp_ones if x not in most_imp_ones2]

['A2_mean_min_diff_flux',
 'A4_max_mean_diff_flux',
 'A5_plus_1_sigma',
 'A5_plus_3_sigma']

In [27]:
most_imp_ones2 = ['A1_minus_3_sigma','A5_max_median_diff_flux','A5_minus_3_sigma','A5_max_mean_diff_flux']

In [28]:
#final_dict2[final_dict2['column_name'].isin(most_imp_ones)]
#final_dict2[final_dict2['fold_sum'] == 1]

In [30]:
#final_dict2.head()

In [ ]:
#final_dict2.to_csv('final_features3.csv',index=False)

In [ ]:
#modify to work with kfold
#def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):
    
        
    sm=SMOTE(random_state=51)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

        
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [ ]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()

temp = temp.merge(train_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')

print(temp.shape)
temp.fillna(0, inplace=True)

y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 1000, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10, 
'min_child_weight': 200.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
'reg_alpha': 0.0, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    trn_xa, trn_y, val_xa, val_y=smoteAdataset(trn_x.values, trn_y.values, val_x.values, val_y.values)
    trn_x=pd.DataFrame(data=trn_xa, columns=trn_x.columns)
    val_x=pd.DataFrame(data=val_xa, columns=val_x.columns)
    
    print(trn_x.shape,trn_y.shape,val_x.shape,val_y.shape)
    
    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

In [31]:
test_metadata['A1_minus_3_sigma'] = test_metadata['A1_mean_flux'] - 3*test_metadata['A1_std_flux']
test_metadata['A5_max_median_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_median_flux']
test_metadata['A5_minus_3_sigma'] = test_metadata['A5_mean_flux'] - 3*test_metadata['A5_std_flux']
test_metadata['A5_max_mean_diff_flux'] = test_metadata['A5_max_flux'] - test_metadata['A5_mean_flux']

In [32]:
train_metadata_kaggle = train_metadata_kaggle.merge(train_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')
test_metadata_kaggle = test_metadata_kaggle.merge(test_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')

In [33]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 104) (3492890, 103)


In [35]:
%%time
train_metadata_kaggle.to_csv('mydata_train_metadata.csv',index=False)
test_metadata_kaggle.to_csv('mydata_test_metadata.csv',index=False)

CPU times: user 7min 29s, sys: 6.05 s, total: 7min 35s
Wall time: 7min 36s


In [ ]:
imp_df.sort_values('gain',ascending=False)

In [ ]:
temp_test = test_metadata_kaggle.copy()

In [ ]:
del temp_test['object_id']

In [ ]:
temp_test.fillna(0.0,inplace = True)

In [ ]:
print(temp.shape,temp_test.shape)

In [ ]:
list(temp.columns) == list(temp_test.columns)

In [ ]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

In [ ]:
test_pred2 = pd.DataFrame()
test_pred2 = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [ ]:
print(test_pred2.shape)

In [ ]:
#test_pred2 = pd.DataFrame(np.random.rand(10,14))

In [ ]:
test_pred2.head()

In [ ]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [ ]:
test_pred2.columns = temp_columns[1:15]

In [ ]:
def getUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

In [ ]:
klm = pd.DataFrame()
klm['mymean'] = test_pred2[feats].mean(axis=1)
klm['mymedian'] = test_pred2[feats].median(axis=1)
klm['mymax'] = test_pred2[feats].max(axis=1)

In [ ]:
test_pred2['class_99'] = getUnknown(klm)

In [ ]:
test_pred2.tail()

In [ ]:
test_pred2 = test_pred2.reset_index(drop=True)

In [ ]:
print(test_pred2.shape,test_id.shape)

In [ ]:
test_id.tail()

In [ ]:
test_id = test_id.reset_index(drop=True)

In [ ]:
test_id.index == test_pred2.index

In [ ]:
%%time
test_pred = pd.concat([test_id,test_pred2],axis=1)

In [ ]:
test_pred = test_pred[temp_columns]

In [ ]:
test_pred.head()

In [ ]:
print(test_pred.shape)

In [ ]:
%%time
test_pred.to_csv('test_pred_20.csv',index=False)

In [ ]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_20.csv -m "Message"